### A First Example

Suppose that we want to compute the sum $ \sum_{i=0}^n \alpha^i $ for given $ \alpha, n $.

Suppose further that we’ve forgotten the basic formula

$$
\sum_{i=0}^n \alpha^i = \frac{1 - \alpha^{n+1}}{1 - \alpha}
$$

for a geometric progression and hence have resolved to rely on a loop.

#### Python vs C

Here’s a pure Python function that does the job

In [ ]:
def geo_prog(alpha, n):
    current = 1.0
    sum = current
    for i in range(n):
        current = current * alpha
        sum = sum + current
    return sum

In [ ]:
%%timeit -n1 -r1 
geo_prog(0.99, int(10**6))

This works fine but for large $ n $ it is slow.

Here’s a C function that will do the same thing

If you’re not familiar with C, the main thing you should take notice of is the
type definitions

- `int` means integer  
- `double` means double precision floating-point number  
- the `double` in `double geo_prog(...` indicates that the function will
  return a double  


Not surprisingly, the C code is faster than the Python code.

#### A Cython Implementation

Cython implementations look like a convex combination of Python and C.

We’re going to run our Cython code in the Jupyter notebook, so we’ll start by
loading the Cython extension in a notebook cell

In [ ]:
%load_ext Cython

In [ ]:
%%cython -a
def geo_prog_cython(double alpha, int n):
    cdef double current = 1.0
    cdef double sum = current
    cdef int i
    for i in range(n):
        current = current * alpha
        sum = sum + current
    return sum

Here `cdef` is a Cython keyword indicating a variable declaration and is followed by a type.

The `%%cython` line at the top is not actually Cython code — it’s a Jupyter cell magic indicating the start of Cython code.

After executing the cell, you can now call the function `geo_prog_cython` from within Python.

In [ ]:
%%timeit -n1 -r1 
geo_prog_cython(0.99, int(10**6))

### Example 2: Cython with NumPy Arrays

Let’s go back to the first problem that we worked with: generating the iterates of the quadratic map

$$
x_{t+1} = 4 x_t (1 - x_t)
$$

The problem of computing iterates and returning a time series requires us to work with arrays.

The natural array type to work with is NumPy arrays.

Here’s a Cython implementation that initializes, populates and returns a NumPy
array

In [ ]:
%%cython -a
import numpy as np

def qm_cython_first_pass(double x0, int n):
    cdef int t
    x = np.zeros(n+1, float)
    x[0] = x0
    for t in range(n):
        x[t+1] = 4.0 * x[t] * (1 - x[t])
    return np.asarray(x)

In [ ]:
%%timeit -n1 -r1 
qm_cython_first_pass(0.1, int(10**5))

The reason is that working with NumPy arrays incurs substantial Python overheads.

We can do better by using Cython’s [typed memoryviews](http://docs.cython.org/src/userguide/memoryviews.html), which provide more direct access to arrays in memory.

When using them, the first step is to create a NumPy array.

Next, we declare a memoryview and bind it to the NumPy array.

Here’s an example:

In [ ]:
%%cython -a
import numpy as np
from numpy cimport float_t

def qm_cython(double x0, int n):
    cdef int t
    x_np_array = np.zeros(n+1, dtype=float)
    cdef float_t [:] x = x_np_array
    x[0] = x0
    for t in range(n):
        x[t+1] = 4.0 * x[t] * (1 - x[t])
    return np.asarray(x)

Here

- `cimport` pulls in some compile-time information from NumPy  
- `cdef float_t [:] x = x_np_array` creates a memoryview on the NumPy array `x_np_array`  
- the return statement uses `np.asarray(x)` to convert the memoryview back to a NumPy array  


Let’s time it:

In [ ]:
%%timeit -n1 -r1 
qm_cython(0.1, int(10**5))